# msFeaST example OMSW comparison between Pleurotus 0 and 80 percent omsw samples

This Jupyter notebook collects all the steps needed to run the msFeaST pipeline the illustrative example and produce the dashboard json file.

In the first block, all dependencies are loaded. All specified packages should be available after installing msFeaST as instructed.

In [ ]:
%load_ext autoreload
%autoreload 2
from matchms.importing import load_from_mgf
from matchms.exporting import save_as_mgf
import pandas as pd
import os
import msfeast.pipeline
import os
import pandas as pd

In this second block all relative filepaths are specified. This includes the input and output paths. When using ms2deepscore, a model path may need to be specified in addition (see corresponding notebook).

*<span style="color:magenta">Required user input: Make sure that all relative file paths are specified correctly.</span>*

In [ ]:
print("Define Filepaths...")
test_data_directory = os.path.join("data", "omsw_pleurotus_comparison")
filepath_test_spectra = os.path.join(test_data_directory, "spectra.mgf")
filepath_test_quant_table = os.path.join(test_data_directory, "quant_table.csv")
filepath_test_treat_table = os.path.join(test_data_directory, "treat_table.csv")

output_directory = os.path.join(test_data_directory)
r_output_filename = os.path.join("r_output.json")
r_filepath = os.path.join(output_directory, r_output_filename)
dashboard_output_filepath = os.path.join(output_directory, "dashboard_data.json")

Loading data and initializing pipeline.

In [ ]:
print("Initializing pipeline...")
pipeline = msfeast.pipeline.Msfeast()

print("Attaching data...")
treat_table = pd.read_csv(filepath_test_treat_table)
quant_table = pd.read_csv(filepath_test_quant_table)
pipeline.attach_spectra_from_file(filepath_test_spectra, identifier_key="scans")
pipeline.attach_quantification_table(quant_table)
pipeline.attach_treatment_table(treat_table)


The spectral similarity computations are run using the provided score.

In [ ]:
print("Running spectral similarity computations...")
pipeline.run_and_attach_spectral_similarity_computations("ModifiedCosine")

*<span style="color:magenta">Required user input: Specify the values of k to trial run using k-medoid clustering.</span>*

In [ ]:
print("Run kmedoid grid...")
pipeline.run_and_attach_kmedoid_grid([50,100,150,200,250])

*<span style="color:magenta">Required user input: Select an appropriate value of k using its iloc, balancing silhouette score and desired number of clusters.</span>*

In [ ]:
pipeline.select_kmedoid_settings(iloc = 4)

*<span style="color:magenta">Required user input: Select perplexity values to trial run.</span>*

In [ ]:
print("Run t-sne grid...")
pipeline.run_and_attach_tsne_grid([20, 30, 40, 50, 100])

*<span style="color:magenta">Required user input: Select appropriate pexplexity score using its iloc.</span>*
High pearson and spearman scores indicate a good correspondence between high dimensional and low dimensional distance. However, distance preservation should be balanced against good grouping qualities in the embedding.

In [ ]:
pipeline.select_tsne_settings(iloc = 3)

Run the statistics routine and integrate all pieces of information into output json file.

In [ ]:
print("Initializing R runtime...")
if os.path.isfile(r_filepath):
  os.remove(r_filepath)
pipeline.run_and_attach_statistical_comparisons(output_directory, r_output_filename)

print("Integrating pipeline results...")
pipeline.integrate_and_attach_dashboard_data(top_k_max=50, alpha=0.01)

print("Exporting json file...")
pipeline.export_dashboard_json(filepath=dashboard_output_filepath)

print("Processing complete.")